Now that I've streamlined the MCMC process, I am going to submit multiple chains simultaneously. This notebook will make multiple, similar config files, for broad comparison. 

This may be rolled into pearce as a helper function, I haven't decided.

For rmin 0, 0.5, 1.0:

    For no ab, HSAB and CorrAB emu:

        Vpeak sham
        Mpeak sham
        HOD
        HSAB HOD 

In [1]:
import yaml
import copy
from os import path
import numpy as np

In [2]:
orig_cfg_fname = '/u/ki/swmclau2/Git/pearce/bin/mcmc/nh_gg_sham_hsab_mcmc_config.yaml'
with open(orig_cfg_fname, 'r') as yamlfile:
    orig_cfg = yaml.load(yamlfile)

In [3]:
bsub_template="""#BSUB -q long 
#BSUB -W 72:00
#BSUB -J {jobname}
#BSUB -oo /u/ki/swmclau2/Git/pearce/bin/mcmc/config/{jobname}.out    
#BSUB -n 8 
#BSUB -R "span[ptile=8]"

python /u/ki/swmclau2/Git/pearce/pearce/inference/initialize_mcmc.py {jobname}.yaml
python /u/ki/swmclau2/Git/pearce/pearce/inference/run_mcmc.py {jobname}.yaml
"""

In [4]:
r_bins = np.logspace(-1, 1.6, 19)

In [5]:
emu_names = ['HOD','HSAB','CorrAB']
emu_fnames = [['/u/ki/swmclau2/des/wp_zheng07/PearceWpCosmo.hdf5', '/u/ki/swmclau2/des/ds_zheng07/PearceDsCosmo.hdf5'],\
              ['/u/ki/swmclau2/des/wp_hsab/PearceWpHSABCosmo.hdf5', '/u/ki/swmclau2/des/ds_hsab/PearceDsHSABCosmo.hdf5'],
              ['/u/ki/swmclau2/des/wp_corrab/PearceWpCorrABCosmo.hdf5', '/u/ki/swmclau2/des/ds_corrab/PearceDsCorrABCosmo.hdf5']]


In [6]:
meas_cov_fname = '/u/ki/swmclau2/Git/pearce/bin/covmat/wp_ds_full_covmat.npy'

# TODO replace with actual ones onace test boxes are done 
emu_cov_fnames = [['/u/ki/swmclau2/Git/pearce/bin/optimization/wp_hod_emu_cov.npy', 
                   '/u/ki/swmclau2/Git/pearce/bin/optimization/ds_hod_emu_cov.npy'],
                  ['/u/ki/swmclau2/Git/pearce/bin/optimization/wp_hsab_emu_cov.npy', 
                   '/u/ki/swmclau2/Git/pearce/bin/optimization/ds_hsab_emu_cov.npy'],
                 ['/u/ki/swmclau2/Git/pearce/bin/optimization/wp_corrab_emu_cov.npy',
                  '/u/ki/swmclau2/Git/pearce/bin/optimization/ds_corrab_emu_cov.npy']]

In [7]:
n_walkers = 200
n_steps = 50000

## Vpeak SHAM
----


In [8]:
tmp_cfg = copy.deepcopy(orig_cfg)
directory = "/afs/slac.stanford.edu/u/ki/swmclau2/Git/pearce/bin/mcmc/config/"
output_dir = "/nfs/slac/g/ki/ki18/des/swmclau2/PearceMCMC/"
jobname_template = "VpeakSHAM_wp_ds_rmin_{rmin}_{emu_name}"

for rmin in [None, 0.5, 1.0, 2.0]:
    for emu_fname, emu_name, emu_cov in zip(emu_fnames, emu_names, emu_cov_fnames):
        
        if rmin is not None:
            tmp_cfg['emu']['fixed_params'] = {'z': 0.0, 'rmin':rmin}
            
        tmp_cfg['emu']['training_file'] = emu_fname
        tmp_cfg['emu']['emu_type'] = ['NashvilleHot' for i in xrange(len(emu_fname))]
        tmp_cfg['emu']['emu_cov_fname'] = emu_cov
        
        tmp_cfg['data']['true_data_fname']= ['/u/ki/swmclau2/Git/pearce/bin/mock_measurements/SHAMmock_wp.npy',
                                             '/u/ki/swmclau2/Git/pearce/bin/mock_measurements/SHAMmock_ds.npy']
        tmp_cfg['data']['true_cov_fname'] = meas_cov_fname
        
        tmp_cfg['data']['obs']['obs'] = ['wp','ds']
        tmp_cfg['data']['obs']['rbins'] = list(r_bins)
        

        tmp_cfg['chain']['nsteps'] = n_steps
        tmp_cfg['chain']['nwalkers'] = n_walkers
        tmp_cfg['chain']['mcmc_type'] = 'normal'

        
        tmp_cfg['data']['sim']['sim_hps']['system'] = 'ki-ls'
        tmp_cfg['data']['cov']['emu_cov_fname'] = tmp_cfg['emu']['emu_cov_fname'] 
        tmp_cfg['data']['cov']['meas_cov_fname'] = tmp_cfg['data']['true_cov_fname']
        
        jobname = jobname_template.format(rmin=rmin, emu_name=emu_name)
        tmp_cfg['fname'] = path.join(output_dir, jobname+'.hdf5')

        with open(path.join(directory, jobname +'.yaml'), 'w') as f:
            yaml.dump(tmp_cfg, f)

        with open(path.join(directory, jobname + '.bsub'), 'w') as f:
            f.write(bsub_template.format(jobname=jobname))

## Shuffled SHAM
----


In [9]:
tmp_cfg = copy.deepcopy(orig_cfg)
directory = "/afs/slac.stanford.edu/u/ki/swmclau2/Git/pearce/bin/mcmc/config/"
output_dir = "/nfs/slac/g/ki/ki18/des/swmclau2/PearceMCMC/"
jobname_template = "ShuffledSHAM_wp_ds_rmin_{rmin}_{emu_name}"

for rmin in [None, 0.5, 1.0, 2.0]:
    for emu_fname, emu_name, emu_cov in zip(emu_fnames, emu_names, emu_cov_fnames):
        
        if rmin is not None:
            tmp_cfg['emu']['fixed_params'] = {'z': 0.0, 'rmin':rmin}
            
        tmp_cfg['emu']['training_file'] = emu_fname
        tmp_cfg['emu']['emu_type'] = ['NashvilleHot' for i in xrange(len(emu_fname))]
        tmp_cfg['emu']['emu_cov_fname'] = emu_cov
        
        tmp_cfg['data']['true_data_fname']= ['/u/ki/swmclau2/Git/pearce/bin/mock_measurements/SHUFFLED_SHAMmock_wp.npy',
                                             '/u/ki/swmclau2/Git/pearce/bin/mock_measurements/SHUFFLED_SHAMmock_ds.npy']
        tmp_cfg['data']['true_cov_fname'] = meas_cov_fname
        
        tmp_cfg['data']['obs']['obs'] = ['wp','ds']
        tmp_cfg['data']['obs']['rbins'] = list(r_bins)
        

        tmp_cfg['chain']['nsteps'] = n_steps
        tmp_cfg['chain']['nwalkers'] = n_walkers
        tmp_cfg['chain']['mcmc_type'] = 'normal'

        
        tmp_cfg['data']['sim']['sim_hps']['system'] = 'ki-ls'
        tmp_cfg['data']['cov']['emu_cov_fname'] = tmp_cfg['emu']['emu_cov_fname'] 
        tmp_cfg['data']['cov']['meas_cov_fname'] = tmp_cfg['data']['true_cov_fname']
        
        jobname = jobname_template.format(rmin=rmin, emu_name=emu_name)
        tmp_cfg['fname'] = path.join(output_dir, jobname+'.hdf5')

        with open(path.join(directory, jobname +'.yaml'), 'w') as f:
            yaml.dump(tmp_cfg, f)

        with open(path.join(directory, jobname + '.bsub'), 'w') as f:
            f.write(bsub_template.format(jobname=jobname))

## Universe Machine
----


In [10]:
tmp_cfg = copy.deepcopy(orig_cfg)
directory = "/afs/slac.stanford.edu/u/ki/swmclau2/Git/pearce/bin/mcmc/config/"
output_dir = "/nfs/slac/g/ki/ki18/des/swmclau2/PearceMCMC/"
jobname_template = "UniverseMachine_wp_ds_rmin_{rmin}_{emu_name}"

for rmin in [None, 0.5, 1.0, 2.0]:
    for emu_fname, emu_name, emu_cov in zip(emu_fnames, emu_names, emu_cov_fnames):
        
        if rmin is not None:
            tmp_cfg['emu']['fixed_params'] = {'z': 0.0, 'rmin':rmin}
            
        tmp_cfg['emu']['training_file'] = emu_fname
        tmp_cfg['emu']['emu_type'] = ['NashvilleHot' for i in xrange(len(emu_fname))]
        tmp_cfg['emu']['emu_cov_fname'] = emu_cov
        
        tmp_cfg['data']['true_data_fname']= ['/u/ki/swmclau2/Git/pearce/bin/mock_measurements/UMmock_wp.npy',
                                             '/u/ki/swmclau2/Git/pearce/bin/mock_measurements/UMmock_ds.npy']
        tmp_cfg['data']['true_cov_fname'] = meas_cov_fname
        
        tmp_cfg['data']['obs']['obs'] = ['wp','ds']
        tmp_cfg['data']['obs']['rbins'] = list(r_bins)
        

        tmp_cfg['chain']['nsteps'] = n_steps
        tmp_cfg['chain']['nwalkers'] = n_walkers
        tmp_cfg['chain']['mcmc_type'] = 'normal'

        
        tmp_cfg['data']['sim']['sim_hps']['system'] = 'ki-ls'
        tmp_cfg['data']['cov']['emu_cov_fname'] = tmp_cfg['emu']['emu_cov_fname'] 
        tmp_cfg['data']['cov']['meas_cov_fname'] = tmp_cfg['data']['true_cov_fname']
        
        jobname = jobname_template.format(rmin=rmin, emu_name=emu_name)
        tmp_cfg['fname'] = path.join(output_dir, jobname+'.hdf5')

        with open(path.join(directory, jobname +'.yaml'), 'w') as f:
            yaml.dump(tmp_cfg, f)

        with open(path.join(directory, jobname + '.bsub'), 'w') as f:
            f.write(bsub_template.format(jobname=jobname))

## HOD
----


In [11]:
#orig_cfg_fname = '/u/ki/swmclau2//Git/pearce/bin/mcmc/nh_gg_sham_hsab_mcmc_config.yaml'
with open(orig_cfg_fname, 'r') as yamlfile:
    orig_cfg = yaml.load(yamlfile)

In [12]:
tmp_cfg = copy.deepcopy(orig_cfg)
directory = "/u/ki/swmclau2/Git/pearce/bin/mcmc/config/"
output_dir = "/nfs/slac/g/ki/ki18/des/swmclau2/PearceMCMC/"
#output_dir = "/afs/slac.stanford.edu/u/ki/swmclau2"

jobname_template = "HOD_wp_ds_rmin_{rmin}_{emu_name}"#_fixed_HOD"

for rmin in [None, 0.5, 1.0, 2.0]:
    for emu_fname, emu_name, emu_cov in zip(emu_fnames, emu_names, emu_cov_fnames):
        
        if rmin is not None:
            tmp_cfg['emu']['fixed_params'] = {'z': 0.0, 'rmin':rmin}
            
        tmp_cfg['emu']['training_file'] = emu_fname
        tmp_cfg['emu']['emu_type'] = ['NashvilleHot' for i in xrange(len(emu_fname))]
        tmp_cfg['emu']['emu_cov_fname'] = emu_cov
        
        tmp_cfg['data']['obs']['obs'] = ['wp','ds']
        tmp_cfg['data']['obs']['rbins'] = list(r_bins)
        
        tmp_cfg['data']['cov']['meas_cov_fname'] = meas_cov_fname
        tmp_cfg['data']['cov']['emu_cov_fname'] = tmp_cfg['emu']['emu_cov_fname'] # TODO make this not be redundant
        
        jobname = jobname_template.format(rmin=rmin, emu_name=emu_name)
        tmp_cfg['fname'] = path.join(output_dir, jobname+'.hdf5')
        
        tmp_cfg['sim']= {'gal_type': 'HOD',
       'hod_name': 'zheng07',
       'hod_params': {'alpha': 1.083,
        'logM0': 13.2,
        'logM1': 14.2,
        'sigma_logM': 0.2,
        'conc_gal_bias': 1.0},
       'nd': '5e-4',
       'scale_factor': 1.0,
       'min_ptcl': 100, 
       'sim_hps': {'boxno': 1,
        'downsample_factor': 1e-2,
        'particles': True,
        'realization': 0,
        'system': 'ki-ls'},
       'simname': 'testbox'}
        
        # TODO i shouldnt have to specify this this way
        tmp_cfg['data']['sim'] = tmp_cfg['sim']
        
        tmp_cfg['chain']['nwalkers'] = n_walkers
        tmp_cfg['chain']['nsteps'] = n_steps
        tmp_cfg['chain']['mcmc_type'] = 'normal'
        
        # fix params during MCMC    
        #tmp_cfg['chain']['fixed_params'].update(tmp_cfg['sim']['hod_params'])
        
        try:
            del tmp_cfg['data']['true_data_fname']
            del tmp_cfg['data']['true_cov_fname']
        except KeyError:
            pass

        with open(path.join(directory, jobname +'.yaml'), 'w') as f:
            yaml.dump(tmp_cfg, f)

        with open(path.join(directory, jobname + '.bsub'), 'w') as f:
            f.write(bsub_template.format(jobname=jobname))

## HSAB HOD
----
